<a href="https://colab.research.google.com/github/ty875/BeautifulSoup/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load in necessary libraries

In [ ]:
import requests #pip install requests
from bs4 import BeautifulSoup as bs #pip install beautifulSoup4


# Load our first page

In [ ]:
#Load the webpage content
r = requests.get("https://ty875.github.io/BeautifulSoup/")

#Convert to a beautiful soup object
soup = bs(r.content, "html.parser")

#Print out the html 
print(soup.prettify())

# Start Scraping

#### find and find_all

In [ ]:
first_header = soup.find("h2")

headers = soup.find_all("h2") #a list of elements
print(headers)

In [ ]:
#Pass in a list of elements to look for
first_header = soup.find(["h2","h1"])

headers = soup.find_all(["h1","h2"])
headers

In [ ]:
#pass in attributes to the find/find_all
paragraph = soup.find_all("p", attrs={"id": "paragraph-id"})
paragraph

In [ ]:
#nesting find/find_all calls
body = soup.find('body')
div = body.find('div')
header = div.find('h1')
header

In [ ]:
#search for specific string in find/find_all calls
print(soup)
import re
paragraphs = soup.find_all("p", string = re.compile("This"))
paragraphs